In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

# Загрузим данные
 * При первом запуске скачаем train.csv и evaluation.csv
 * При повторных запусках файлы уже будут на месте

In [2]:
data = pd.DataFrame.from_csv("./train.csv")

In [ ]:
# data = data.loc[data['cancel_time']<5000]
X = data[data.columns[:len(data.columns)-1]]

In [3]:
y = data['burned']

In [ ]:
test = pd.DataFrame.from_csv("./test.csv")

In [ ]:
# len(X[X['cancel_time']<0])*1./len(X)
# X = X.loc[X['cancel_time']<500]
# plt.figure(figsize=(16, 8))
# X_notminusone = X.loc[X['cancel_time']!=-1]
# trsh = plt.hist(np.log(np.array([1.]*len(X_notminusone)) + X_notminusone['cancel_time'].values), bins=2000)
# plt.xlim(0., 7.)
# plt.ylim(0., 3000.)

In [ ]:
# plt.figure(figsize=(16, 8))
# trash = plt.hist( data.loc[(data['burned']==1) & (data['cancel_time']<100000)]['cancel_time'], bins=1000 )
# plt.ylim(0, 5000)
# plt.xlim(0, 100000)

In [ ]:
# data.loc[(data['burned']==0)]['cancel_time'].value_counts()

In [ ]:
# data.loc[(data['burned']==1)]['cancel_time'].value_counts()

In [ ]:
# test.head(1)

# citycenters

In [ ]:
#!!!!!
# X = pd.DataFrame.from_csv("./train_with_cityid.csv")

In [ ]:
centers = [
    [55.754216, 37.61343],  #0
    [51.661535, 39.200287], #1
    [56.326887, 44.005986], #2
    [55.798551, 49.106324], #3
    [59.939095, 30.315868]  #4
]

from geopy.distance import vincenty

minminmin = 10**15

def city_id(coords):
    mn = 0
    min_value = minminmin
    for i in range(len(centers)):
        
        dist = vincenty(coords, centers[i]).km
        if (dist < min_value):
            min_value = dist
            mn = i
    if (min_value > 10**3):
        return -1
    return mn

In [ ]:
XV = X.values
X['city_id'] = [city_id([x[5], x[6]]) for x in XV]

In [ ]:
X.to_csv('train_with_cityid.csv')

In [ ]:
# X = pd.read_csv('train_with_cityid.csv')

In [ ]:
testV = test.values
test['city_id'] = [city_id([x[3], x[4]]) for x in testV]

In [ ]:
test.to_csv('test_with_cityid.csv')

In [ ]:
# test = pd.read_csv('test_with_cityid.csv')

# concat

In [ ]:
print len(X.loc[X['driver_found']==True])
print len(y[y==False])


In [ ]:
x_col = ['dist', 'due', 'lat', 'lon', 'f_class', 's_class', 't_class', 'city_id']
X_and_test = pd.concat([X[x_col], test[x_col]])

# Preprocessing

In [ ]:
#Используем данные о праздниках отсюда - http://eduscan.net/help/calendar2014 http://newslab.ru/article/559455

holidays = "1.01,2.01,3.01,4.01,5.01,6.01,7.01,8.01,"\
"23.02,8.03,9.03,10.03,1.05,2.05,3.05,4.05,9.05,10.05,11.05,12.06,13.06".split(',')

holidays = map(lambda s: tuple(map(int,s.split('.'))),holidays)

#вытаскиватель категориальных фичей
from sklearn.feature_extraction import DictVectorizer
vectorizer = DictVectorizer(sparse=False,dtype=np.bool)

# используем встроенный питоновый парсер времени
from dateutil import parser

def preprocess_data(X_raw):

    #распарсим время
    datetimes = list(X_raw.due.apply(lambda x: parser.parse(x)))

    #время дня от 0 до 1
    rel_times = map(lambda dt: (dt.hour*60 + dt.minute)/(24.*60), datetimes)

    #день недели
    week_days = map( lambda x: x.isoweekday() , datetimes)

    rel_times = np.array(rel_times)
    week_days = np.array(week_days)
    
    is_holiday = map(lambda dt: (dt.day,dt.month) in holidays,
                 datetimes)

    dow_names = ['mon','tue','wed','thu','fri','sat','sun']
    
    
    data_dict = ( {'city_id':str(city_id), 
                   'f_class':f,
                   's_class':(s if (pd.isnull(s)) else (str(f) + str(s))),
                   't_class':(t if (pd.isnull(t)) else(str(f) + str(s) + str(t))),
                   'day_of_week':dow_names[day_of_week-1]}
                 
            for (city_id,f,s,t),day_of_week in zip(X_raw[['city_id', 
                                                         'f_class',
                                                         's_class',
                                                         't_class']].values, week_days) )

    
    Xcat = vectorizer.fit_transform(data_dict)

    other_features = ["dist","lat","lon"]
    
    Xreal = X_raw[other_features].values
    
    
    Xfull = np.concatenate([
            Xreal,
            Xcat            
        ],axis=1)
    
    Xfull = pd.DataFrame(Xfull,columns=other_features+vectorizer.feature_names_)
    
    Xfull["time_of_day_rel"] = rel_times
    Xfull["is_holiday"] = is_holiday

    
    return Xfull

In [ ]:
X_and_test = preprocess_data(X_and_test)

In [ ]:
X_and_test.to_csv('trainplustest_preprocessed.csv')

# [ если всё упало ]

In [ ]:
X_and_test = pd.read_csv('trainplustest_preprocessed.csv')

In [ ]:
# y = data['burned'].values
X = X_and_test.head(len(y))
test = X_and_test[len(y):]

In [ ]:
print X.shape, y.shape, test.shape
print data.shape

# предскажем cancel_time

### лол, cancel_time, не считая -1, имеет логнормальное распределение, значит его имеет смысл предсказывать

In [ ]:
cancel_times = data['cancel_time']

In [ ]:
print X.shape, cancel_times.shape

In [ ]:
cancel_times = np.log(cancel_times + np.array([2.]*len(cancel_times)))

In [ ]:
trash = plt.hist(cancel_times[cancel_times != 0], bins=100)

In [ ]:
params = {} 
params['objective']           = "reg:linear"
# params['booster']             = "gbtree"
# params['eval_metric']         = "auc"
params['eta']                 = 0.1
# params['max_depth']           = 7
# params['subsample']           = 0.681
# params['colsample_bytree']    = 0.95
params['seed']                = 1337
params['njobs']               = -1
# params['silent']              = 1
bound = int(X.shape[0]*0.9)
x_train, x_val = X[:bound], X[bound:]
y_train, y_val = cancel_times[:bound], cancel_times[bound:]
 
dtrain = xgb.DMatrix(x_train, y_train)
dval = xgb.DMatrix(x_val, y_val)
# dtest = xgb.DMatrix(test)
watchlist = [(dval,'eval')]

In [ ]:
num_trees = 300
esr = 10
gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, verbose_eval=True, early_stopping_rounds = esr)

In [ ]:
gbm.best_ntree_limit

In [ ]:
dtrain = xgb.DMatrix(X, cancel_times)
dtest = xgb.DMatrix(test)

In [ ]:
gbm = xgb.train(params, dtrain, 25)

In [ ]:
pred_cancel_time = gbm.predict(dtest, ntree_limit=25)

In [ ]:
pd.DataFrame(pred_cancel_time).to_csv('predicted_cancel_times.csv')

In [ ]:
dtrain_ = xgb.DMatrix(X)
train_pred_cancel_time = gbm.predict(dtrain_, ntree_limit=25)

In [ ]:
pd.DataFrame(train_pred_cancel_time).to_csv('train_predicted_cancel_times.csv')

In [ ]:
###
pred_cancel_times = pd.read_csv('predicted_cancel_times.csv')
train_pred_cancel_times = pd.read_csv('train_predicted_cancel_times.csv')

### ! Теперь у нас есть pred_cancel_time для test!!!!!

# предскажем driver_found

In [ ]:
driver_founds = data['driver_found']

In [ ]:
params = {} 
params['objective']           = "binary:logistic"
# params['booster']             = "gbtree"
# params['eval_metric']         = "auc"
params['eta']                 = 0.05
# params['max_depth']           = 7
# params['subsample']           = 0.681
# params['colsample_bytree']    = 0.95
params['seed']                = 1337
params['njobs']               = -1
# params['silent']              = 1
bound = int(X.shape[0]*0.5)
x_train, x_val = X[:bound], X[bound:]
y_train, y_val = driver_founds[:bound], driver_founds[bound:]
 
dtrain = xgb.DMatrix(x_train, y_train)
dval = xgb.DMatrix(x_val, y_val)
# dtest = xgb.DMatrix(test)
watchlist = [(dval,'eval')]

In [ ]:
num_trees = 300
esr = 10
gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, verbose_eval=True, early_stopping_rounds = esr)

In [ ]:
gbm.best_ntree_limit

In [ ]:
dtrain = xgb.DMatrix(X, driver_founds)
dtest = xgb.DMatrix(test)

In [ ]:
gbm = xgb.train(params, dtrain, 28)

In [ ]:
pred_driver_founds = gbm.predict(dtest, ntree_limit=28)

In [ ]:
pd.DataFrame(pred_driver_founds).to_csv('predicted_driver_founds.csv')

In [ ]:
dtrain_ = xgb.DMatrix(X)
train_pred_driver_founds = gbm.predict(dtrain_, ntree_limit=25)

In [ ]:
pd.DataFrame(train_pred_driver_founds).to_csv('train_predicted_driver_founds.csv')

In [ ]:
###
pred_driver_founds = pd.read_csv('predicted_driver_founds.csv')
train_pred_driver_founds = pd.read_csv('train_predicted_driver_founds.csv')

# append cancels and drivers

### for train

In [ ]:
train_pred_cancel_times.index = X.index
del train_pred_cancel_times['Unnamed: 0']
X['cancel_times'] = train_pred_cancel_times
train_pred_driver_founds.index = X.index
del train_pred_driver_founds['Unnamed: 0']
X['driver_found'] = train_pred_driver_founds
X.head(1)

### for test

In [ ]:
pred_cancel_times.index = test.index
del pred_cancel_times['Unnamed: 0']
test['cancel_times'] = pred_cancel_times
pred_driver_founds.index = test.index
del pred_driver_founds['Unnamed: 0']
test['driver_found'] = pred_driver_founds
test.head(1)

In [ ]:
X.to_csv('X_with_metas.csv')
test.to_csv('test_with_metas.csv')

# если всё пошло по пизде

In [ ]:
X = pd.read_csv('X_with_metas.csv')
test = pd.read_csv('test_with_metas.csv')

# добавим vectorized кластеры

### в train

In [ ]:
# Xapp1 = X[:10].copy()
# Xapp2 = X[10:20].copy()
# Xapp2.index = Xapp1.index

# Xres = pd.concat([Xapp1, Xapp2[['lat','lon']]], axis=1,join_axes=[Xapp1.index])
# Xres

In [ ]:
X.head()

In [ ]:
trainclusters = pd.read_csv('train_clusters_vectorized.csv')

In [ ]:
del trainclusters['Unnamed: 0']
trainclusters.head()

In [ ]:
X = pd.concat([X, trainclusters], axis=1, join_axes=[X.index])

In [ ]:
X.head()

### в test

In [ ]:
test.head()

In [ ]:
testclusters.head()

In [ ]:
testclusters = pd.read_csv('test_clusters_vectorized.csv')

In [ ]:
del testclusters['Unnamed: 0']

In [ ]:
test = pd.concat([test, testclusters], axis=1, join_axes=[test.index])

In [ ]:
test.head()

# LEARN

In [ ]:
del X['Unnamed: 0']
del X['Unnamed: 0.1']
X.head()

In [ ]:
del test['Unnamed: 0']
del test['Unnamed: 0.1']
test.head()

In [ ]:
X.to_csv('X_clust_meta.csv')
test.to_csv('test_clust_meta.csv')

In [4]:
X = pd.read_csv('X_clust_meta.csv')
test = pd.read_csv('test_clust_meta.csv')

In [6]:
del X['Unnamed: 0']
X.head()

,dist,lat,lon,city_id=-1,city_id=0,city_id=1,city_id=2,city_id=3,city_id=4,day_of_week=fri,...,cluster_id=90,cluster_id=91,cluster_id=92,cluster_id=93,cluster_id=94,cluster_id=95,cluster_id=96,cluster_id=97,cluster_id=98,cluster_id=99
0,17990.125431,55.750130,37.823242,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,17990.125431,55.750130,37.823242,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,11439.391945,55.651582,37.340891,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,14608.577392,55.633404,37.797595,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,6070.886393,55.770330,37.519917,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [8]:
del test['Unnamed: 0']
test.head()

,dist,lat,lon,city_id=-1,city_id=0,city_id=1,city_id=2,city_id=3,city_id=4,day_of_week=fri,...,cluster_id=90,cluster_id=91,cluster_id=92,cluster_id=93,cluster_id=94,cluster_id=95,cluster_id=96,cluster_id=97,cluster_id=98,cluster_id=99
0,1748.201178,55.731713,37.487353,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,False,False,True,False,False,False,False,False,False,False
1,7341.870806,55.973637,37.412352,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,False,False,False,False,False,False,False,False,False,False
2,7398.857088,55.743193,37.427839,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,False,False,False,False,False,False,False,False,False,False
3,11787.335702,55.777737,37.617626,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,False,False,False,False,False,False,False,False,False,False
4,4418.580801,55.785083,37.589114,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [14]:
# del y['Unnamed: 0']
y = pd.DataFrame(y)

In [18]:
y.index = np.arange(0, len(y))

In [23]:
y = pd.Series(y['burned'])

In [25]:
params = {} 
params['objective']           = "binary:logistic" 
params['booster']             = "gbtree"
params['eval_metric']         = "auc"
params['eta']                 = 0.006
params['max_depth']           = 7
params['subsample']           = 0.681
params['colsample_bytree']    = 0.95
params['seed']                = 18
params['njobs']               = -1
params['silent']              = 1

In [26]:
X.shape, y.shape, test.shape

((1793300, 266), (1793300,), (743463, 266))

In [ ]:
bound = int(X.shape[0]*0.9)
x_train, x_val = X[:bound], X[bound:]
y_train, y_val = y[:bound], y[bound:]
 
dtrain = xgb.DMatrix(x_train, y_train)
dval = xgb.DMatrix(x_val, y_val)
dtest = xgb.DMatrix(test)
watchlist = [(dval,'eval')]

In [ ]:
num_trees = 5000
esr = 12
gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, verbose_eval=True, early_stopping_rounds = esr)
gbm.best_ntree_limit

In [30]:
dtrain = xgb.DMatrix(X, y)
dtest = xgb.DMatrix(test)

In [ ]:
gbm = xgb.train(params, dtrain, 2000)
gbm.save_model('clust_meta.model')

In [ ]:
prediction = gbm.predict(dtest, ntree_limit=2000)

# save

In [ ]:
response = pd.DataFrame()
response["Ids"] = np.arange(test.shape[0])
response["Y_prob"] = prediction
 
response.to_csv('stacksub.csv',index=None)

In [ ]:
# prediction = gbm.predict(dtrain, ntree_limit=780)

In [ ]:
# response = pd.DataFrame()
# response["Ids"] = np.arange(X.shape[0])
# response["Y_prob"] = prediction
 
# response.to_csv('train779.csv',index=None)

In [ ]:
# %%time
from sklearn.ensemble import RandomForestClassifier

#Также обучим более "случайную" версию RandomForest
model = RandomForestClassifier(n_estimators=779,n_jobs=16,min_samples_split=75,min_samples_leaf=20)

model.fit(X, y)

In [ ]:
res = model.predict_proba(test)

In [ ]:
# res = res[:, 1]

response = pd.DataFrame()
response["Ids"] = np.arange(test.shape[0])
response["Y_prob"] = res
 
response.to_csv('rf_test779.csv',index=None)

In [ ]:
res = model.predict_proba(X)

# res = res[:, 1]

response = pd.DataFrame()
response["Ids"] = np.arange(X.shape[0])
response["Y_prob"] = res
 
response.to_csv('rf_train779.csv',index=None)

In [ ]:
np.corr(res, prediction)

In [ ]:
# from datanight import plot_forest_feature_importances
# plot_forest_feature_importances(model,X.columns)

In [ ]:
# from datanight import generate_submission
# generate_submission(preprocess_data,model,"submission.csv")

# Советы по улучшению модели:

* Смотри глазами на данные - строй графики, исследуй аномалия 
 * это может принести новые идеи и отбросить нежизнеспособные намного раньше 
 * Туториал по библиотеке, в которой можно строить графики 
   * http://matplotlib.org/users/pyplot_tutorial.html
 * Ключевой вопрос самому себе - __"От чего ещё может зависеть, сгорит ли заказ?"__


* Попробуй более точно настроить модель или выбрать другую
 * Random Forest с текущими параметрами можно улучшить
   * Документация по нему - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
 * А можно использовать другие, более подходящие методы
   * Документация по методам - http://scikit-learn.org/stable/supervised_learning.html
 * Наконец, модели можно комбинировать
   * Например, усреднять с весами
   * В итоге качество может быть лучше, чем у каждой модели по отдельности
 * Важно не увлечься - зачастую качественные изменения решения намного лучше подкручивания параметров


* Подумай, какие ещё дополнительные данные можно провязать с выборкой? 
 * насколько реально применить их за отведённое время?


* __Главное__ - пытайся понять, дадут ли твои улучшения прирост на новых данных
 * Тестовая выборка находится по времени дальше, чем обучающая.
   * Например, нельзя использовать номер дня в году, потому что мы обучаемся на выборке __ДО__ начала контрольной
 * Это не значит, что нужно отправлять миллион решений на проверку и выбирать лучшее - это приведёт к переобучению
 